# The TikTok-to-Spotify Pipeline
**TXC Group X**<br>
Leticia Brendle - 70033 <br>
X<br>
X<br>

## Research Question
"Does TikTok virality create a 'popularity ceiling' effect? Investigating how TikTok engagement patterns predict and limit long-term streaming success across Spotify."



### Hypotheses

H1: Songs with extremely high TikTok engagement (top 10%) show diminishing returns on Spotify long-term streaming compared to moderate TikTok performers

H2: The TikTok-Spotify conversion rate follows an inverted U-shape, with optimal TikTok engagement existing in the middle range

Notes:

Methods: Polynomial regression to test non-linear relationships + threshold analysis for identifying optimal TikTok engagement levels

Why Novel: Tests the counterintuitive idea that TikTok success might actually limit rather than enhance long-term success
Business Relevance: Critical for music industry investment and artist development strategies

## 3. Data Set Up

### 3.1 Import Libraries and Data
First we will import all the necessary libraries as well as our data sets. We will also have a first look at the structure of the data to help us understand it.

In [10]:
# Import libraries
# Data
import pandas as pd 
import numpy as np

# Graphs
import seaborn as sns
import matplotlib.pyplot as plt

# Statistics
import scipy.stats as stats
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson

# Load CSV file 
songs_spotify = pd.read_csv('songsspotify.csv', encoding='cp1252')

## Provide Data Overview ## 
print("First have a look at the first few rows of our data set and the general structure: \n")
print(songs_spotify.head())
print("\n\nNumber of Rows and Columns: " + str(songs_spotify.shape) + "\n") 



UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfd in position 2679: invalid start byte

## 3.2 Data Cleaning

clean data


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset laden
df = pd.read_csv('songsspotify.csv', encoding='cp1252')
print(f"Original dataset: {df.shape[0]} rows, {df.shape[1]} columns")

# =============================================================================
# 3.3 DATA CLEANING
# =============================================================================

# 3.3.1 DROP UNNECESSARY COLUMNS
print("\n3.3.1 Drop Unnecessary Columns")
print("-" * 40)

# Für TikTok-to-Spotify Pipeline relevante Spalten
relevant_columns = [
    'Track', 'Artist', 'Release Date',
    'Spotify Streams', 'Spotify Popularity', 'Spotify Playlist Count', 
    'TikTok Posts', 'TikTok Likes', 'TikTok Views',
    'Track Score', 'All Time Rank'
]

print(f"Keeping {len(relevant_columns)} relevant columns from {df.shape[1]} total columns:")
for col in relevant_columns:
    print(f"  • {col}")

# Unnötige Spalten droppen
df_clean = df[relevant_columns].copy()
print(f"\nDataset after dropping columns: {df_clean.shape[0]} rows, {df_clean.shape[1]} columns")

# 3.3.2 MISSING VALUES
print("\n3.3.2 Missing Values")
print("-" * 40)

# Missing values analysieren
missing_summary = df_clean.isnull().sum()
missing_percentage = (missing_summary / len(df_clean)) * 100

print("Missing values per column:")
for col in df_clean.columns:
    missing_count = missing_summary[col]
    missing_pct = missing_percentage[col]
    print(f"  {col}: {missing_count} ({missing_pct:.1f}%)")

# Numerische Spalten identifizieren und Kommas entfernen
numeric_columns = [
    'Spotify Streams', 'Spotify Popularity', 'Spotify Playlist Count',
    'TikTok Posts', 'TikTok Likes', 'TikTok Views', 
    'Track Score', 'All Time Rank'
]

# String-Zahlen zu numerisch konvertieren (Kommas entfernen)
for col in numeric_columns:
    if col in df_clean.columns:
        # Kommas entfernen und zu numerisch konvertieren
        df_clean[col] = df_clean[col].astype(str).str.replace(',', '').str.replace(' ', '')
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Release Date zu datetime
df_clean['Release Date'] = pd.to_datetime(df_clean['Release Date'], errors='coerce')

# Rows mit missing values droppen
print(f"\nRows before dropping missing values: {len(df_clean)}")
df_clean = df_clean.dropna()
print(f"Rows after dropping missing values: {len(df_clean)}")

# Nochmal nach missing values checken
print(f"\nDouble-checking for missing values after dropping:")
remaining_missing = df_clean.isnull().sum()
total_missing = remaining_missing.sum()

for col in df_clean.columns:
    if remaining_missing[col] > 0:
        print(f"  {col}: {remaining_missing[col]} missing values")
    
if total_missing == 0:
    print("✅ No missing values remaining!")
else:
    print(f"⚠️ Still {total_missing} missing values found!")
    print("Dropping remaining rows with any missing values...")
    df_clean = df_clean.dropna()
    print(f"Final rows after complete cleaning: {len(df_clean)}")
    
    # Final check
    final_missing = df_clean.isnull().sum().sum()
    if final_missing == 0:
        print("✅ All missing values successfully removed!")
    else:
        print(f"❌ Still {final_missing} missing values remaining")

# 3.3.3 DUPLICATES
print("\n3.3.3 Duplicates")
print("-" * 40)

# Duplicates basierend auf Track + Artist identifizieren
print("Checking for duplicates based on Track + Artist combination...")

duplicates_before = df_clean.duplicated(subset=['Track', 'Artist']).sum()
print(f"Duplicate songs found: {duplicates_before}")

if duplicates_before > 0:
    print("\nExamples of duplicate songs:")
    duplicate_songs = df_clean[df_clean.duplicated(subset=['Track', 'Artist'], keep=False)]
    duplicate_examples = duplicate_songs.groupby(['Track', 'Artist']).size().head()
    for (track, artist), count in duplicate_examples.items():
        print(f"  '{track}' by {artist}: {count} entries")
    
    # Duplicates droppen (erste Occurrence behalten)
    rows_before_dup_removal = len(df_clean)
    df_clean = df_clean.drop_duplicates(subset=['Track', 'Artist'], keep='first')
    duplicates_removed = rows_before_dup_removal - len(df_clean)
    print(f"\nDuplicates removed: {duplicates_removed}")
else:
    print("✅ No duplicates found!")

# Nochmal nach duplicates checken
duplicates_after = df_clean.duplicated(subset=['Track', 'Artist']).sum()
print(f"Double-checking for duplicates after removal: {duplicates_after}")
print(f"Final dataset: {df_clean.shape[0]} rows, {df_clean.shape[1]} columns")

# =============================================================================
# CLEANING SUMMARY
# =============================================================================
print("\n" + "="*50)
print("DATA CLEANING SUMMARY")
print("="*50)
print(f"Original dataset: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"After dropping columns: {len(relevant_columns)} columns kept")
print(f"After handling missing values: {len(df_clean)} rows remaining")
print(f"After removing duplicates: {len(df_clean)} unique songs")
print(f"Final missing values: {df_clean.isnull().sum().sum()}")

print(f"\nDataset ready for TikTok-to-Spotify Pipeline analysis!")
print(f"Key variables available:")
print(f"  • TikTok Metrics: Posts, Likes, Views")
print(f"  • Spotify Metrics: Streams, Popularity, Playlist Count")
print(f"  • Control Variables: Track Score, All Time Rank, Release Date")

# Quick data quality check
songs_with_both_data = ((df_clean['TikTok Posts'] > 0) | (df_clean['TikTok Likes'] > 0)) & (df_clean['Spotify Streams'] > 0)
print(f"\nSongs with both TikTok and Spotify data: {songs_with_both_data.sum()} ({songs_with_both_data.mean():.1%})")

# Save cleaned dataset
df_clean.to_csv('spotify_cleaned.csv', index=False)
print(f"\nCleaned dataset saved as 'spotify_cleaned.csv'")
print("Ready for Exploratory Data Analysis! 🚀")

Original dataset: 4600 rows, 29 columns

3.3.1 Drop Unnecessary Columns
----------------------------------------
Keeping 11 relevant columns from 29 total columns:
  • Track
  • Artist
  • Release Date
  • Spotify Streams
  • Spotify Popularity
  • Spotify Playlist Count
  • TikTok Posts
  • TikTok Likes
  • TikTok Views
  • Track Score
  • All Time Rank

Dataset after dropping columns: 4600 rows, 11 columns

3.3.2 Missing Values
----------------------------------------
Missing values per column:
  Track: 0 (0.0%)
  Artist: 5 (0.1%)
  Release Date: 0 (0.0%)
  Spotify Streams: 113 (2.5%)
  Spotify Popularity: 804 (17.5%)
  Spotify Playlist Count: 70 (1.5%)
  TikTok Posts: 1173 (25.5%)
  TikTok Likes: 980 (21.3%)
  TikTok Views: 981 (21.3%)
  Track Score: 0 (0.0%)
  All Time Rank: 0 (0.0%)

Rows before dropping missing values: 4600
Rows after dropping missing values: 3171

Double-checking for missing values after dropping:
✅ No missing values remaining!

3.3.3 Duplicates
----------------